In [97]:
import os
import time
import tqdm
import datetime
import pickle as pkl
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset

In [1]:
## Make sure the following folders have been included in the working director:
## Best_Trained: which contains the best word embedding learned from the best logistic regression and neural net models
## Best_Pretrained: which contains the word embedding learned of pre-trained embeddings
## pickle: which will include the following two pickle files: 1) 10000_id2token.pkl 2) 10000_token2id.pkl

In [98]:
import load_data

In [99]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

In [129]:
## Set up emvironment
CURR_PATH = os.getcwd()
WEIGHT_PATH = '/Best_Trained/'
PRETRAINED_WEIGHT_PATH = '/Best_Pretrained/'
VOCAB_SIZE = 10000

### for 3.1

In [123]:
## Load trained weight
Best_lr = torch.load(CURR_PATH + WEIGHT_PATH + '10000_50_MUL_log-reg.pt')
Best_nn = torch.load(CURR_PATH + WEIGHT_PATH + '10000_500_MUL_neural-net.pt')

embedding_lr = Best_lr['encoder.embed.weight']
embedding_nn = Best_nn['encoder.embed.weight']

In [124]:
## laod token_index loop up tables
id2token = pkl.load(open('pickle/'+str(VOCAB_SIZE)+'_id2token.pkl', 'rb'))
token2id = pkl.load(open('pickle/'+str(VOCAB_SIZE)+'_token2id.pkl', 'rb'))

In [125]:
## Calculate similarity matrix
simi_lr = cosine_similarity(embedding_lr)
simi_nn = cosine_similarity(embedding_nn)

/Users/haonantian/anaconda3/envs/nlpclass/lib/python3.6/site-packages/sklearn/utils/extmath.py:91: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/haonantian/anaconda3/envs/nlpclass/lib/python3.6/site-packages/sklearn/utils/extmath.py:91: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)


In [126]:
def expand(simi_matrix, k=10):
    '''
    Expand the similarity matrix to a sorted array with similarity socres
    '''
    res = []
    for i in range(len(simi_matrix)-1):
        for j in range(i+1,len(simi_matrix[0])):
            if simi_matrix[i][j] != 0 and simi_matrix[i][j] <= 0.99:
                res.append(simi_matrix[i][j])
        if i % 500 == 0:
            print('finished {}'.format(i))
    print('Done expanding')
    return sorted(res)

def generate_word_pair(matrix, threshold, id2_token):
    '''
    Generate top 10 similar tokens with the 
    '''
    res = []
    for i in range(len(matrix)-1):
        for j in range(i+1, len(matrix[0])):
            if matrix[i][j] >= threshold:
                res.append([id2_token[i], id2_token[j]])
        if i % 500 == 0:
            print('finished {}'.format(i))
    print('Done token pair generation')
    return res

In [127]:
## Generate sorted array of similarity scores
sorted_lr = expand(simi_lr)
sorted_nn = expand(simi_nn)

finished 0
finished 500
finished 1000
finished 1500
finished 2000
finished 2500
finished 3000
finished 3500
finished 4000
finished 4500
finished 5000
finished 5500
finished 6000
finished 6500
finished 7000
finished 7500
finished 8000
finished 8500
finished 9000
finished 9500
Done expanding
finished 0
finished 500
finished 1000
finished 1500
finished 2000
finished 2500
finished 3000
finished 3500
finished 4000
finished 4500
finished 5000
finished 5500
finished 6000
finished 6500
finished 7000
finished 7500
finished 8000
finished 8500
finished 9000
finished 9500
Done expanding


In [128]:
## Generate top 10 most similar token pairs
res_lr = generate_word_pair(simi_lr, sorted_lr[-10], id2token)
res_nn = generate_word_pair(simi_nn, sorted_nn[-10], id2token)

finished 0
finished 500
finished 1000
finished 1500
finished 2000
finished 2500
finished 3000
finished 3500
finished 4000
finished 4500
finished 5000
finished 5500
finished 6000
finished 6500
finished 7000
finished 7500
finished 8000
finished 8500
finished 9000
finished 9500
Done token pair generation
finished 0
finished 500
finished 1000
finished 1500
finished 2000
finished 2500
finished 3000
finished 3500
finished 4000
finished 4500
finished 5000
finished 5500
finished 6000
finished 6500
finished 7000
finished 7500
finished 8000
finished 8500
finished 9000
finished 9500
Done token pair generation


In [130]:
## top 10 similar token pairs for linear regression
res_lr

[['under', 'wakeboard'],
 ['worker', 'entertains'],
 ['Nobody', 'shower'],
 ['sad', 'joyously'],
 ['swims', 'cooler'],
 ['great', 'Island'],
 ['bars', 'sanded'],
 ['snowcapped', 'intimate'],
 ['gliding', 'beaming'],
 ['sponge', 'mountaintops']]

In [131]:
## top 10 similar token pairs for neural network
res_nn

[['sleeping', 'Nobody'],
 ['sleeping', 'sleeps'],
 ['sleeping', 'movies'],
 ['no', 'Nobody'],
 ['lunch', 'hour'],
 ['birthday', 'joyously'],
 ['happily', 'joyously'],
 ['cats', 'sleep'],
 ['conference', 'Hispanic'],
 ['siblings', 'joyously']]

### for 3.4

In [135]:
## Load pre-trained weight
Best_pretrained = torch.load(CURR_PATH + PRETRAINED_WEIGHT_PATH + '10000_pretrained_DIRECT_neural-net.pt')
embedding_pre = Best_pretrained['encoder.embed.weight']

In [138]:
## Calculate similarity matrix
pretrained_simi = cosine_similarity(embedding_pre)

/Users/haonantian/anaconda3/envs/nlpclass/lib/python3.6/site-packages/sklearn/utils/extmath.py:91: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)


In [158]:
## Generate sorted array of similarities
sorted_pre = expand(pretrained_simi)

finished 0
finished 500
finished 1000
finished 1500
finished 2000
finished 2500
finished 3000
finished 3500
finished 4000
finished 4500
finished 5000
finished 5500
finished 6000
finished 6500
finished 7000
finished 7500
finished 8000
finished 8500
finished 9000
finished 9500
Done expanding


In [166]:
## Generate top 10 similar token pairs for pre-trained embedding
res_pretrained = generate_word_pair(pretrained_simi, sorted_pre[-10], id2token)

finished 0
finished 500
finished 1000
finished 1500
finished 2000
finished 2500
finished 3000
finished 3500
finished 4000
finished 4500
finished 5000
finished 5500
finished 6000
finished 6500
finished 7000
finished 7500
finished 8000
finished 8500
finished 9000
finished 9500
Done token pair generation


In [167]:
## Present the result for pretrained embedding
res_pretrained

[['13', '14'],
 ['14', '15'],
 ['14', '16'],
 ['14', '19'],
 ['19', '23'],
 ['19', '21'],
 ['22', '23'],
 ['22', '21'],
 ['53', '56'],
 ['23', '21']]